In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#%matplotlib inline
import numpy as np
import pandas as pd

import sys

from skimage.io import imread
from matplotlib import pyplot as plt
import random

import keras
from keras import models
from keras.optimizers import SGD
from keras import models
from keras.layers import Input
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
import os,cv2

# Load the data

In [31]:
X = []
Y = []

dir_ = '/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/'

for im_name in os.listdir(dir_+'dataset_1/LAGdataset_200/'):
  print(im_name)
  if os.path.isfile(dir_+'masks_1/'+im_name+'_mask.png'):
    img  = cv2.imread(dir_+'dataset_1/LAGdataset_200/'+im_name)
    img = cv2.resize(img,(128,128))
    X.append(img)
    mask = cv2.imread(dir_+'masks_1/'+im_name+'_mask.png',0)
    #print(mask.shape)
    Y.append(mask)
    #break
    #print(im_name)



Childhood model_2.png
Luglio_00415_1.png
Breaking_Bad_00684_1.png
How_I_met_your_mother_00675_1.png
claudio-marchisio_1.png
20-KatherineHeigl-jpg_111136_2.png
Gennaio_01576_2.png
8430907_6_2.png
celebs-then-now-young-old-22_2.png
then-and-now-the-cast-of-grease-photos-8_2.png
l43-robbie-williams-130712125227_big_2.png
Agosto_01539_1.png
Christian-Bale_2.png
kim-kardashian-viso_1.png
C_2_fotogallery_1007624__ImageGallery__imageGalleryItem_6_image_2.png
Signore_Anelli_e_Hobbit_01172_2.png
1382085236_092_1.png
Hunger_Games_00705_2.png
549bc37f3e659_-_elle-11-sara-sampaio-h_3_add.png
jeff-cohen-ieri_2.png
Pd2zM1N_1.png
22.-Russell-Brand_1.png
audrey_hepburn_e203d5c0040f027bd0908db67ce07f9c_1_add.png
jamie-foxx_2.png
celebs-then-now-young-old-63_1.png
celebs-then-now-young-old-5_1.png
Politicians_00391_1.png
a025b5a4633d2fedcc7f4d14f948d952_1.png
Aprile_01641_2.png
Aprile_04152_1.png
1288173906_doseng.org_1288153636_002_2.png
1320414856_kak-molody-oni-byli_33824_s__2_1.png
Settembre_03661_2

In [34]:
X = np.array(X)
print(X.shape)
X = np.reshape(X,(X.shape[0],X.shape[1],X.shape[2],X.shape[3]))
Y = np.array(Y)
print(Y.shape)
Y = np.reshape(Y,(Y.shape[0],Y.shape[1],Y.shape[2],1))
print(X.shape, Y.shape)

(996, 128, 128, 3)
(996, 128, 128, 1)
(996, 128, 128, 3) (996, 128, 128, 1)


# Normalize Input and Output.


In [0]:
X = X/255.0
Y =Y/255.0


In [0]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

# Define Unet Model architecture

In [0]:
def UNet():
    f = [64, 128, 256, 512, 1024]
    inputs = keras.layers.Input((128, 128, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [83]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_157 (Conv2D)             (None, 128, 128, 64) 1792        input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_158 (Conv2D)             (None, 128, 128, 64) 36928       conv2d_157[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_35 (MaxPooling2D) (None, 64, 64, 64)   0           conv2d_158[0][0]                 
____________________________________________________________________________________________

# Train the model

In [0]:

model.fit(x=X, y= Y, batch_size=32, shuffle = True, validation_split =0.1,epochs=40)

# Save the model

In [49]:

model_json = model.to_json()
with open("/content/drive/My Drive/mask_rcnn_segmentation/model_40.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("/content/drive/My Drive/mask_rcnn_segmentation/model_40.h5")
print("Saved model to disk")

Saved model to disk


In [70]:
from keras.models import Model,model_from_json
json_file = open('/content/drive/My Drive/mask_rcnn_segmentation/unet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/mask_rcnn_segmentation/unet.h5")
print("Loaded model from disk")

Loaded model from disk


# Save the masks

In [73]:
img_dir = '/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/images/'
for i in os.listdir(img_dir):
  print(i)
  image = cv2.imread(img_dir+i)
  image = cv2.resize(image,(128,128))
  image = np.reshape(image,(1,128,128,3))
  mask = loaded_model.predict(image)
  mask = np.reshape(mask,(128,128))
  mask = mask.astype('uint8')
  image = np.reshape(image,(128,128,3))
  seg_image = np.zeros(shape =image.shape)
  seg_image[:,:,0] = np.multiply(image[:,:,0], mask)
  seg_image[:,:,1] = np.multiply(image[:,:,1], mask)
  seg_image[:,:,2] = np.multiply(image[:,:,2], mask)
  mask[mask==1] = 255

  cv2.imwrite('/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/results_unet/'+str(i)+'_mask.png', mask)

  cv2.imwrite('/content/drive/My Drive/mask_rcnn_segmentation/keras-mask-rcnn/results_unet/'+str(i)+'_seg.png', seg_image)

5.jpg
(128, 128)
9.jpg
(128, 128)
2.jpg
(128, 128)
3.jpg
(128, 128)
10.jpg
(128, 128)
8.jpg
(128, 128)
4.jpg
(128, 128)
1.jpg
(128, 128)
6.jpg
(128, 128)
7.jpg
(128, 128)
